In [1]:
import os
import pickle
from skimage.io import imread
from skimage.transform import resize
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,mean_absolute_error, classification_report, confusion_matrix, average_precision_score
from sklearn.preprocessing import StandardScaler

# Data loading
input_dir = r"C:\Users\Dell\Desktop\ParkEase\clf-data"
categories = ['empty', 'not_empty']
data = []
labels = []

for category_idx, category in enumerate(categories):
    for file in os.listdir(os.path.join(input_dir, category)):
        img_path = os.path.join(input_dir, category, file)
        img = imread(img_path)
        img = resize(img, (15, 15))
        data.append(img.flatten())
        labels.append(category_idx)

data = np.asarray(data)
labels = np.asarray(labels)

# Train/test split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.5, shuffle=True, stratify=labels)

# Feature scaling (important for logistic regression)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Train Logistic Regression classifier
classifier = LogisticRegression(max_iter=1000)
parameters = [{'C': [0.1, 1, 10, 100], 'solver': ['liblinear', 'lbfgs']}]
grid_search = GridSearchCV(classifier, parameters, cv=5)
grid_search.fit(x_train_scaled, y_train)

# Test performance
best_estimator = grid_search.best_estimator_
print("Best parameters found:", grid_search.best_params_)
y_prediction = best_estimator.predict(x_test_scaled)
score = accuracy_score(y_test, y_prediction)
print('{}% of samples were correctly classified'.format(str(score * 100)))

mae = mean_absolute_error(y_test, y_prediction)
absolute_errors = np.abs(y_test - y_prediction)
sae = np.std(absolute_errors)
print("Mean Absolute Error (MAE):", mae)
print("Standard Absolute Error (SAE):", sae)
# Detailed metrics
print("Precision:", precision_score(y_test, y_prediction))
print("Recall:", recall_score(y_test, y_prediction))
print("F1 Score:", f1_score(y_test, y_prediction))
print("Average Precision:", average_precision_score(y_test, y_prediction))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_prediction))
print("\nClassification Report:\n", classification_report(y_test, y_prediction))

Best parameters found: {'C': 1, 'solver': 'lbfgs'}
99.13513513513513% of samples were correctly classified
Mean Absolute Error (MAE): 0.008648648648648649
Standard Absolute Error (SAE): 0.09259508369887073
Precision: 1.0
Recall: 0.9842416283650689
F1 Score: 0.9920582395764395
Average Precision: 0.9928902770137176
Confusion Matrix:
 [[1252    0]
 [  24 1499]]

Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      1252
           1       1.00      0.98      0.99      1523

    accuracy                           0.99      2775
   macro avg       0.99      0.99      0.99      2775
weighted avg       0.99      0.99      0.99      2775

